## Tutorial 17 - Navier Stokes equations
**_Keywords: DEIM, supremizer operator_**

### 1. Introduction

In this tutorial, we will study the Navier-Stokes equations over a two-dimensional backward-facing step domain $\Omega$.

A Poiseuille flow profile is imposed on the inlet boundary, and a no-flow (zero velocity) condition is imposed on the walls. A homogeneous Neumann condition of the Cauchy stress tensor is applied at the outflow boundary.

The inflow velocity boundary condition is characterized by $$\boldsymbol{u}(\boldsymbol{x};\mu)=\mu\bigg \{\frac{1}{2.25}(x_1-2)(5-x_1),0\bigg \} \quad \forall \boldsymbol{x}=(x_0,x_1) \in \Omega$$ 

This problem is characterized by one parameter $\mu$, which characterizes the inlet velocity. The range of $\mu$ is the following $$\mu \in [1.0, 80.0].$$ 

Thus, the parameter domain is $$\mathbb{P}=[1.0,80.0].$$

In order to obtain a faster approximation of the problem, we pursue a model reduction by means of a POD-Galerkin reduced order method.


### 2. Parametrized formulation

Let $\boldsymbol{u}(\mu)$ be the velocity vector and $p(\mu)$ be the pressure in the domain $\Omega$.

We will directly provide a weak formulation for this problem: for a given parameter $\mu \in \mathbb{P},$ find $u(\mu) \in \mathbb{V}(\mu), \; p \in\mathbb{M}$ such that
    
<center>
    $
    \begin{cases}
        \nu \int_{\Omega} \nabla \boldsymbol{u} : \nabla \boldsymbol{v} \ d\Omega + \int_{\Omega} [(\boldsymbol{u} \cdot \nabla) \boldsymbol{u}] \cdot \boldsymbol{v} \ d\Omega - \int_{\Omega} p \nabla \cdot \boldsymbol{v} \ d\Omega = \int_{\Omega} \boldsymbol{f} \cdot \boldsymbol{v} \ d\Omega, \quad \forall \boldsymbol{v} \in\mathbb{V},  \\
        \int_{\Omega} q \nabla \cdot \boldsymbol{u} \ d\Omega = 0, \quad \forall q \in\mathbb{M}
    \end{cases}
    $
</center>    
where

* $\nu$ represents kinematic viscosity
* the functional space $\mathbb{V}(\mu)$ is defined as $\mathbb{V}=[H^1_{\Gamma_{wall}}(\Omega)]^2$
* the functional space $\mathbb{M}(\mu)$ is defined as $\mathbb{M}=L^2(\Omega)$


Since this problem utilizes mixed finite element discretization with the velocity and pressure as solution variables, the inf-sup condition is necessary for the well posedness of this problem. Thus, the supremizer operator $T^{\mu}: \mathbb{M}_h \rightarrow \mathbb{V}_h$ will be used.


In [1]:
import os
import sys
sys.path.append('../../')

from mlnics import NN, Losses, Normalization, RONNData, IO, Training, ErrorAnalysis
from dolfin import *
from rbnics import *
from ufl import transpose
import torch
import numpy as np

torch.manual_seed(0)
np.random.seed(0)

### 3. Affine Decomposition 



In [2]:
@DEIM("online", basis_generation="Greedy")
@ExactParametrizedFunctions("offline")
class NavierStokes(NavierStokesProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        NavierStokesProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        dup = TrialFunction(V)
        (self.du, self.dp) = split(dup)
        (self.u, _) = split(self._solution)
        vq = TestFunction(V)
        (self.v, self.q) = split(vq)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)
        #
        self.inlet = Expression(("1. / 2.25 * (x[1] - 2) * (5 - x[1])", "0."), degree=2)
        self.f = Constant((0.0, 0.0))
        self.g = Constant(0.0)
        # Customize nonlinear solver parameters
        self._nonlinear_solver_parameters.update({
            "linear_solver": "mumps",
            "maximum_iterations": 20,
            "report": True
        })

    # Return custom problem name
    def name(self):
        return "NavierStokesDEIM1"

    # Return theta multiplicative terms of the affine expansion of the problem.
    @compute_theta_for_derivatives
    @compute_theta_for_supremizers
    def compute_theta(self, term):
        mu = self.mu
        if term == "a":
            theta_a0 = 1.
            return (theta_a0,)
        elif term in ("b", "bt"):
            theta_b0 = 1.
            return (theta_b0,)
        elif term == "c":
            theta_c0 = 1.
            return (theta_c0,)
        elif term == "f":
            theta_f0 = 1.
            return (theta_f0,)
        elif term == "g":
            theta_g0 = 1.
            return (theta_g0,)
        elif term == "dirichlet_bc_u":
            theta_bc00 = mu[0]
            return (theta_bc00,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_derivatives
    @assemble_operator_for_supremizers
    def assemble_operator(self, term):
        dx = self.dx
        if term == "a":
            u = self.du
            v = self.v
            a0 = inner(grad(u) + transpose(grad(u)), grad(v)) * dx
            return (a0,)
        elif term == "b":
            u = self.du
            q = self.q
            b0 = - q * div(u) * dx
            return (b0,)
        elif term == "bt":
            p = self.dp
            v = self.v
            bt0 = - p * div(v) * dx
            return (bt0,)
        elif term == "c":
            u = self.u
            v = self.v
            c0 = inner(grad(u) * u, v) * dx
            return (c0,)
        elif term == "f":
            v = self.v
            f0 = inner(self.f, v) * dx
            return (f0,)
        elif term == "g":
            q = self.q
            g0 = self.g * q * dx
            return (g0,)
        elif term == "dirichlet_bc_u":
            bc0 = [DirichletBC(self.V.sub(0), self.inlet, self.boundaries, 1),
                   DirichletBC(self.V.sub(0), Constant((0.0, 0.0)), self.boundaries, 2)]
            return (bc0,)
        elif term == "inner_product_u":
            u = self.du
            v = self.v
            x0 = inner(grad(u), grad(v)) * dx
            return (x0,)
        elif term == "inner_product_p":
            p = self.dp
            q = self.q
            x0 = inner(p, q) * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")


# Customize the resulting reduced problem
@CustomizeReducedProblemFor(NavierStokesProblem)
def CustomizeReducedNavierStokes(ReducedNavierStokes_Base):
    class ReducedNavierStokes(ReducedNavierStokes_Base):
        def __init__(self, truth_problem, **kwargs):
            ReducedNavierStokes_Base.__init__(self, truth_problem, **kwargs)
            self._nonlinear_solver_parameters.update({
                "report": True,
                "line_search": "wolfe"
            })

    return ReducedNavierStokes

## 4. Main program

### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [3]:
mesh = Mesh("data/backward_facing_step.xml")
subdomains = MeshFunction("size_t", mesh, "data/backward_facing_step_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/backward_facing_step_facet_region.xml")

### 4.2. Create Finite Element Space (Taylor-Hood P2-P1)

In [4]:
element_u = VectorElement("Lagrange", mesh.ufl_cell(), 2)
element_p = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
element = MixedElement(element_u, element_p)
V = FunctionSpace(mesh, element, components=[["u", "s"], "p"])

### 4.3. Allocate an object of the NavierStokes class

In [5]:
problem = NavierStokes(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(1.0, 80.0)]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with a POD-Galerkin method

In [6]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(10, DEIM=20)

### 4.5. Perform the offline phase

#### 4.5.1 Fit Reduction Method

In [7]:
lifting_mu = (1.0,)
problem.set_mu(lifting_mu)
reduction_method.initialize_training_set(100, DEIM=144, sampling=EquispacedDistribution())
reduced_problem = reduction_method.offline()

=                     DEIM preprocessing phase begins for                      =
={ sum_{i_{31}} ([v_0[0], v_0[1]])[i_{31}] * ({ A | A_{i_{29}} = sum_{i_{30}} ([f_10[0], f_10[1]])[i_{30}] * ([{ A | A_{i_{32}} = (grad(f_10))[0, i_{32}] }, { A | A_{i_{33}} = (grad(f_10))[1, i_{33}] }])[i_{29}, i_{30}]  })[i_{31}]  } * dx(<Mesh #0>[everywhere], {}),=
=                                    where                                     =
=f_10 = solution of NavierStokesDEIM1 (exact problem decorator: False, component: (None,))=
=[f_10[0], f_10[1]] = solution of NavierStokesDEIM1 (exact problem decorator: False, component: (0,)), component 0=
=               with id 8ccf219ff8fe6701b1bb914fc354683f65880d45               =

:::::::::::::::::::::::::::::::::::: DEIM 0 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (1.0,)
  0 SNES Function norm 1.259061e+01
  1 SNES Function norm 2.175455e-01
  2 SNES Function norm 1.215619e-04
  3 SNES Function norm 1.002245e-10
PETSc 

  4 SNES Function norm 4.835447e-08
  5 SNES Function norm 2.615370e-13
PETSc SNES solver converged in 5 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 18 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (10.944055944055943,)
  0 SNES Function norm 1.476696e+02
  1 SNES Function norm 2.528873e+01
  2 SNES Function norm 8.644388e-01
  3 SNES Function norm 4.842512e-03
  4 SNES Function norm 8.874008e-08
  5 SNES Function norm 2.635457e-13
PETSc SNES solver converged in 5 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 19 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (11.496503496503495,)
  0 SNES Function norm 1.557015e+02
  1 SNES Function norm 2.786315e+01
  2 SNES Function norm 1.011448e+00
  3 SNES Function norm 6.498275e-03
  4 SNES Function norm 1.566373e-07
  5 SNES Function norm 2.801874e-13
PETSc SNES solver converged in

add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 36 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (20.888111888111887,)
  0 SNES Function norm 3.007698e+02
  1 SNES Function norm 8.977772e+01
  2 SNES Function norm 6.168621e+00
  3 SNES Function norm 1.712099e-01
  4 SNES Function norm 8.353767e-05
  5 SNES Function norm 6.831997e-11
PETSc SNES solver converged in 5 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 37 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (21.44055944055944,)
  0 SNES Function norm 3.098056e+02
  1 SNES Function norm 9.446658e+01
  2 SNES Function norm 6.649670e+00
  3 SNES Function norm 1.951553e-01
  4 SNES Function norm 1.075676e-04
  5 SNES Function norm 1.126416e-10
PETSc SNES solver converged in 5 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 38 ::::::::::::::::::::::::::::::::::::
eval

evaluate parametrized expression at mu = (30.83216783216783,)
  0 SNES Function norm 4.719752e+02
  1 SNES Function norm 1.915310e+02
  2 SNES Function norm 1.831778e+01
  3 SNES Function norm 1.096586e+00
  4 SNES Function norm 3.149295e-03
  5 SNES Function norm 7.643661e-08
  6 SNES Function norm 8.062697e-13
PETSc SNES solver converged in 6 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 55 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (31.384615384615383,)
  0 SNES Function norm 4.820189e+02
  1 SNES Function norm 1.982542e+02
  2 SNES Function norm 1.922044e+01
  3 SNES Function norm 1.188045e+00
  4 SNES Function norm 3.693357e-03
  5 SNES Function norm 1.031492e-07
  6 SNES Function norm 7.832227e-13
PETSc SNES solver converged in 6 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 56 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expressio

evaluate parametrized expression at mu = (40.22377622377622,)
  0 SNES Function norm 6.503523e+02
  1 SNES Function norm 3.211257e+02
  2 SNES Function norm 3.717629e+01
  3 SNES Function norm 3.508757e+00
  4 SNES Function norm 3.304247e-02
  5 SNES Function norm 6.008957e-06
  6 SNES Function norm 1.131810e-12
PETSc SNES solver converged in 6 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 72 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (40.77622377622377,)
  0 SNES Function norm 6.613508e+02
  1 SNES Function norm 3.297670e+02
  2 SNES Function norm 3.852519e+01
  3 SNES Function norm 3.717317e+00
  4 SNES Function norm 3.722489e-02
  5 SNES Function norm 7.481609e-06
  6 SNES Function norm 1.159969e-12
PETSc SNES solver converged in 6 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 73 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression

...

truth solve for mu = (20.94949494949495,)
  0 SNES Function norm 3.017710e+02
  1 SNES Function norm 9.029305e+01
  2 SNES Function norm 6.221010e+00
  3 SNES Function norm 1.737563e-01
  4 SNES Function norm 8.595241e-05
  5 SNES Function norm 7.228896e-11
PETSc SNES solver converged in 5 iterations with convergence reason 2.
supremizer solve for mu = (20.94949494949495,)
update snapshots matrix

###################################### 26 ######################################
truth solve for mu = (21.747474747474747,)
  0 SNES Function norm 3.148496e+02
  1 SNES Function norm 9.712095e+01
  2 SNES Function norm 6.926258e+00
  3 SNES Function norm 2.094876e-01
  4 SNES Function norm 1.233637e-04
  5 SNES Function norm 1.476128e-10
PETSc SNES solver converged in 5 iterations with convergence reason 2.
supremizer solve for mu = (21.747474747474747,)
update snapshots matrix

###################################### 27 ######################################
truth solve for mu = (22.545454545

supremizer solve for mu = (33.717171717171716,)
update snapshots matrix

###################################### 42 ######################################
truth solve for mu = (34.515151515151516,)
  0 SNES Function norm 5.399927e+02
  1 SNES Function norm 2.384726e+02
  2 SNES Function norm 2.481419e+01
  3 SNES Function norm 1.813085e+00
  4 SNES Function norm 8.610596e-03
  5 SNES Function norm 5.012630e-07
  6 SNES Function norm 9.080219e-13
PETSc SNES solver converged in 6 iterations with convergence reason 2.
supremizer solve for mu = (34.515151515151516,)
update snapshots matrix

###################################### 43 ######################################
truth solve for mu = (35.313131313131315,)
  0 SNES Function norm 5.550587e+02
  1 SNES Function norm 2.493015e+02
  2 SNES Function norm 2.637223e+01
  3 SNES Function norm 2.004011e+00
  4 SNES Function norm 1.053802e-02
  5 SNES Function norm 7.292290e-07
  6 SNES Function norm 9.178404e-13
PETSc SNES solver converged in 

  6 SNES Function norm 1.491931e-11
PETSc SNES solver converged in 6 iterations with convergence reason 2.
supremizer solve for mu = (46.484848484848484,)
update snapshots matrix

###################################### 58 ######################################
truth solve for mu = (47.282828282828284,)
  0 SNES Function norm 7.951225e+02
  1 SNES Function norm 4.402109e+02
  2 SNES Function norm 5.651526e+01
  3 SNES Function norm 6.891023e+00
  4 SNES Function norm 1.356033e-01
  5 SNES Function norm 8.027351e-05
  6 SNES Function norm 2.587749e-11
PETSc SNES solver converged in 6 iterations with convergence reason 2.
supremizer solve for mu = (47.282828282828284,)
update snapshots matrix

###################################### 59 ######################################
truth solve for mu = (48.08080808080808,)
  0 SNES Function norm 8.120661e+02
  1 SNES Function norm 4.548700e+02
  2 SNES Function norm 5.899453e+01
  3 SNES Function norm 7.382451e+00
  4 SNES Function norm 1.570020e-

  4 SNES Function norm 1.003387e+00
  5 SNES Function norm 3.007418e-03
  6 SNES Function norm 4.589871e-08
PETSc SNES solver converged in 6 iterations with convergence reason 3.
supremizer solve for mu = (59.25252525252525,)
update snapshots matrix

###################################### 74 ######################################
truth solve for mu = (60.05050505050505,)
  0 SNES Function norm 1.080336e+03
  1 SNES Function norm 7.052260e+02
  2 SNES Function norm 1.037600e+02
  3 SNES Function norm 1.839257e+01
  4 SNES Function norm 1.132653e+00
  5 SNES Function norm 3.729729e-03
  6 SNES Function norm 7.251127e-08
PETSc SNES solver converged in 6 iterations with convergence reason 3.
supremizer solve for mu = (60.05050505050505,)
update snapshots matrix

###################################### 75 ######################################
truth solve for mu = (60.84848484848485,)
  0 SNES Function norm 1.099162e+03
  1 SNES Function norm 7.240224e+02
  2 SNES Function norm 1.072715e+02


  3 SNES Function norm 3.864431e+01
  4 SNES Function norm 5.514586e+00
  5 SNES Function norm 6.120210e-02
  6 SNES Function norm 3.202354e-05
  7 SNES Function norm 4.056758e-12
PETSc SNES solver converged in 7 iterations with convergence reason 2.
supremizer solve for mu = (71.22222222222223,)
update snapshots matrix

###################################### 89 ######################################
truth solve for mu = (72.02020202020202,)
  0 SNES Function norm 1.375110e+03
  1 SNES Function norm 1.016972e+03
  2 SNES Function norm 1.639693e+02
  3 SNES Function norm 4.071300e+01
  4 SNES Function norm 6.131549e+00
  5 SNES Function norm 7.422443e-02
  6 SNES Function norm 4.873551e-05
  7 SNES Function norm 7.601832e-12
PETSc SNES solver converged in 7 iterations with convergence reason 2.
supremizer solve for mu = (72.02020202020202,)
update snapshots matrix

###################################### 90 ######################################
truth solve for mu = (72.81818181818183,)


#### 4.5.2 Train PINN

Given a training set $X_{PINN} = (\boldsymbol{\mu}^{(1)}, \dots, \boldsymbol{\mu}^{(n)})$ of parameters for the PDE, we train a Physics-Informed Neural Network (PINN) $\operatorname{N}_W(\boldsymbol{\mu})$ dependent on the weights and biases $W$ of the network to minimize the loss function

$$\begin{align*}
    L_{PINN}(X_{PINN}; W) &= \frac1n \sum_{i=1}^n \left\|A(\boldsymbol{\mu^{(i)}}) \operatorname{N}_W(\boldsymbol{\mu}^{(i)}) + B^\top(\boldsymbol{\mu}^{(i)})\operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \boldsymbol{f}(\boldsymbol{\mu}^{(i)}) + \boldsymbol{c}(\boldsymbol{\mu}^{(i)})\right\|_2^2\\
        &\hspace{10ex}+ \left\|B(\boldsymbol{\mu}^{(i)})\operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \boldsymbol{g}(\boldsymbol{\mu}^{(i)})\right\|_2^2
    \end{align*}$$

over $W$, where for a given $\boldsymbol{\mu}$, $A(\boldsymbol{\mu})$ is the stiffness matrix, $\boldsymbol{f}(\boldsymbol{\mu})$ is the vector corresponding to the forcing term for the first equation in the system, $\boldsymbol{c}(\boldsymbol{\mu})$ is a vector corresponding to the nonlinear term, $B(\boldsymbol{\mu})$ is the matrix which corresponds to the second equation in the system, and $\boldsymbol{g}(\boldsymbol{\mu})$ is the vector corresponding to the forcing term for the second equation.

In [ ]:
input_normalization_pinn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_pinn = Normalization.StandardNormalization()

pinn_net  = NN.RONN("PINN", problem, reduction_method, n_hidden=2, n_neurons=40)
pinn_loss = Losses.PINN_Loss(pinn_net, output_normalization_pinn)
data      = RONNData.RONNDataLoader(pinn_net, validation_proportion=0.2, 
                                    num_without_snapshots=100)
optimizer = torch.optim.Adam(pinn_net.parameters(), lr=0.001)

pinn_trainer = Training.PINNTrainer(
    pinn_net, data, pinn_loss, optimizer,
    input_normalization_pinn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    pinn_net, data, pinn_trainer, optimizer
)

In [ ]:
pinn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(pinn_trainer, pinn_net)

#### 4.5.3 Train PDNN

Given a training set $X_{PDNN} = ((\boldsymbol{\mu}^{(1)}, \operatorname{HF}(\boldsymbol{\mu}^{(1)})), \dots, (\boldsymbol{\mu}^{(n)}, \operatorname{HF}(\boldsymbol{\mu}^{(n)})))$ of parameter and high fidelity solution pairs for the PDE, we train a Projection-Driven Neural Network (PDNN) $\operatorname{N}_W(\boldsymbol{\mu})$ dependent on the weights and biases $W$ of the network to minimize the loss function
$$L_{PDNN}(X_{PDNN}; W) = \frac1n \sum_{i=1}^n \|\operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \tilde{\operatorname{HF}}(\boldsymbol{\mu}^{(i)})\|_2^2,$$
where for a given $\boldsymbol{\mu}$, $\tilde{\operatorname{HF}}(\boldsymbol{\mu})$ is the projection of $\operatorname{HF}(\boldsymbol{\mu})$ onto the reduced order solution space.

In [ ]:
input_normalization_pdnn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_pdnn = Normalization.StandardNormalization()

pdnn_net  = NN.RONN("PDNN", problem, reduction_method, n_hidden=2, n_neurons=40)
pdnn_loss = Losses.PDNN_Loss(pdnn_net, output_normalization_pdnn)
data      = RONNData.RONNDataLoader(pdnn_net, validation_proportion=0.2)
optimizer = torch.optim.Adam(pdnn_net.parameters(), lr=0.001)

pdnn_trainer = Training.PDNNTrainer(
    pdnn_net, data, pdnn_loss, optimizer,
    input_normalization_pdnn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    pdnn_net, data, pdnn_trainer, optimizer
)

In [ ]:
pdnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(pdnn_trainer, pdnn_net)

#### 4.5.4 Train PRNN

We train a Physics-Reinforced Neural Network (PRNN) $N_W(\boldsymbol{\mu})$ dependnent on the weights and biases $W$ of the network to minimize the loss function

$$L_{PRNN}(X_{PINN}, X_{PDNN}; W) = L_{PINN}(X_{PINN}; W) + \omega L_{PDNN}(X_{PDNN}; W),$$

where $\omega$ is a scaling parameter which can be chosen freely.

In [ ]:
input_normalization_prnn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_prnn = Normalization.StandardNormalization()

omega = 1.
prnn_net  = NN.RONN(f"PRNN_{omega}", problem, reduction_method, n_hidden=2, n_neurons=40)
prnn_loss = Losses.PRNN_Loss(prnn_net, output_normalization_prnn, omega=omega)
data      = RONNData.RONNDataLoader(prnn_net, validation_proportion=0.2,
                                    num_without_snapshots=100)
optimizer = torch.optim.Adam(prnn_net.parameters(), lr=0.001)

prnn_trainer = Training.PRNNTrainer(
    prnn_net, data, prnn_loss, optimizer,
    input_normalization_prnn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    prnn_net, data, prnn_trainer, optimizer
)

In [ ]:
prnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(prnn_trainer, prnn_net, separate=True)

### 4.6. Perform an error analysis

#### 4.6.1 Reduction Method Error Analysis

In [ ]:
reduction_method.initialize_testing_set(16, DEIM=25, sampling=EquispacedDistribution())
reduction_method.error_analysis()

#### 4.6.2 PINN Error Analysis

In [ ]:
test_mu = torch.tensor(reduction_method.testing_set)

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    pinn_net, test_mu, input_normalization_pinn, output_normalization_pinn
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    pinn_net, (10.0,), 
    input_normalization_pinn, output_normalization_pinn, 
    colorbar=True, component="u"
)

#### 4.6.3 PDNN Error Analysis

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    pdnn_net, test_mu, input_normalization_pdnn, output_normalization_pdnn
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    pdnn_net, (10.0,), 
    input_normalization_pdnn, output_normalization_pdnn,
    colorbar=True, component="u"
)

#### 4.6.4 PRNN Error Analysis

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    prnn_net, test_mu, input_normalization_prnn, output_normalization_prnn
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    prnn_net, (10.0,), 
    input_normalization_prnn, output_normalization_prnn,
    colorbar=True, component="u"
)

#### 4.6.5 Neural Network Error Comparison

In [ ]:
nets = dict()
nets["pinn_net"] = pinn_net
nets["pdnn_net"] = pdnn_net
nets["prnn_net"] = prnn_net

input_normalizations = dict()
input_normalizations["pinn_net"] = input_normalization_pinn
input_normalizations["pdnn_net"] = input_normalization_pdnn
input_normalizations["prnn_net"] = input_normalization_prnn

output_normalizations = dict()
output_normalizations["pinn_net"] = output_normalization_pinn
output_normalizations["pdnn_net"] = output_normalization_pdnn
output_normalizations["prnn_net"] = output_normalization_prnn

_ = ErrorAnalysis.error_analysis_by_network(
    nets, test_mu, input_normalizations, output_normalizations, euclidean=False
)

### 4.7. Perform a speedup analysis

In [ ]:
reduction_method.speedup_analysis()